In [19]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, Flatten 
from torch.optim import Adam, SGD

# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [20]:
# Training transformations with augmentation
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Assuming grayscale images
])

# Validation transformations without augmentation
validate_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [51]:
# Load datasets
full_train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=validate_transform)
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=validate_transform)

# Split the training dataset
train_size = int(0.9 * len(full_train_set))
validate_size = len(full_train_set) - train_size
train_set, validate_set = random_split(full_train_set, [train_size, validate_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
validate_loader = DataLoader(validate_set, batch_size=64, shuffle=True)

In [52]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.network = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(32),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(64),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(0.25),   # Dropout with 25% probability
            Flatten(),
            Linear(64 * 7 * 7, 128),  # An intermediate dense layer
            ReLU(inplace=True),
            Dropout(0.25),  # Higher dropout before the final layer
            Linear(128, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.network(x)
        return x

In [53]:
from torchinfo import summary

# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.001)
# defining the loss function
criterion = CrossEntropyLoss()
# defining batch
batch_size = 64
# checking if GPU is available
if torch.cuda.is_available():
    model = model.to('cuda')
    criterion = criterion.to('cuda')

print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters')
summary(model, input_size=(1, 1, 28, 28))

421.834 K parameters


Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [1, 10]                   --
├─Sequential: 1-1                        [1, 10]                   --
│    └─Conv2d: 2-1                       [1, 32, 28, 28]           320
│    └─BatchNorm2d: 2-2                  [1, 32, 28, 28]           64
│    └─ReLU: 2-3                         [1, 32, 28, 28]           --
│    └─MaxPool2d: 2-4                    [1, 32, 14, 14]           --
│    └─Conv2d: 2-5                       [1, 64, 14, 14]           18,496
│    └─BatchNorm2d: 2-6                  [1, 64, 14, 14]           128
│    └─ReLU: 2-7                         [1, 64, 14, 14]           --
│    └─MaxPool2d: 2-8                    [1, 64, 7, 7]             --
│    └─Dropout: 2-9                      [1, 64, 7, 7]             --
│    └─Flatten: 2-10                     [1, 3136]                 --
│    └─Linear: 2-11                      [1, 128]                  401,536
│   

In [54]:
def train(epoch):
    print(f'epoch:{epoch}---------')
    model.train()

    iteration = 0
    for images_batch, labels_batch in train_loader:
        # clearing the Gradients of the model parameters
        optimizer.zero_grad()
        
        # prediction for training and validation set
        output_train = model(images_batch.to('cuda'))
    
        # computing the training and validation loss
        loss_train = criterion(output_train, labels_batch.to('cuda'))
        train_losses.append(loss_train.item())
        
        # computing the updated weights of all the model parameters
        loss_train.backward()
    
        optimizer.step()

        iteration += 1
        if iteration%100 == 0:
            print(f'Iteration : {iteration+1}, train loss:{loss_train.item():.4f}')

In [55]:
n_epochs = 10
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

epoch:0---------
Iteration : 101, train loss:0.4613
Iteration : 201, train loss:0.5249
Iteration : 301, train loss:0.5186
Iteration : 401, train loss:0.2951
Iteration : 501, train loss:0.2569
Iteration : 601, train loss:0.3582
Iteration : 701, train loss:0.4523
Iteration : 801, train loss:0.2440
epoch:1---------
Iteration : 101, train loss:0.1507
Iteration : 201, train loss:0.3635
Iteration : 301, train loss:0.2803
Iteration : 401, train loss:0.2411
Iteration : 501, train loss:0.2371
Iteration : 601, train loss:0.2731
Iteration : 701, train loss:0.3043
Iteration : 801, train loss:0.3512
epoch:2---------
Iteration : 101, train loss:0.4662
Iteration : 201, train loss:0.2810
Iteration : 301, train loss:0.2669
Iteration : 401, train loss:0.1754
Iteration : 501, train loss:0.2241
Iteration : 601, train loss:0.1619
Iteration : 701, train loss:0.2234
Iteration : 801, train loss:0.1672
epoch:3---------
Iteration : 101, train loss:0.2732
Iteration : 201, train loss:0.3347
Iteration : 301, train

In [58]:
validate_loader_single_batch = DataLoader(test_set, batch_size=len(test_set), shuffle=False)

# Function to retrieve all data in one batch
def get_all_data_single_batch(dataloader):
    all_images, all_labels = next(iter(dataloader))  # Retrieve the single batch
    return all_images, all_labels

# Retrieve all images and labels
all_validate_images, all_validate_labels = get_all_data_single_batch(validate_loader_single_batch)

with torch.no_grad():
    output_val = model(all_validate_images.to('cuda'))

softmax = torch.exp(output_val).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)
val_accu = accuracy_score(all_validate_labels, predictions)
val_accu

0.9088

In [59]:
with torch.no_grad():
    output_val = model(all_validate_images.to('cuda'))
    loss_val = criterion(output_val, all_validate_labels.to('cuda'))
loss_val.item()

0.2621687948703766

In [18]:
val_accu, loss_val.item()

(0.9125, 0.2625797390937805)